In [5]:
from ortools.graph import pywrapgraph
import time

def conjure_data():
  cost = [[90, 76, 75, 'NA'],
          [35, 85, 55, 65],
          [125, 95, 90, 105],
          [45, 110, 95, 115]]
  return cost

def main():
    cost = conjure_data()
    n_workers = len(cost)
    n_tasks = len(cost[0])
    
    assignment = pywrapgraph.LinearSumAssignment()
    
    for worker in range(n_workers):
        for task in range(n_tasks):
            if cost[worker][task]!='NA':
                assignment.AddArcWithCost(worker, task, cost[worker][task])
                
    solve_status = assignment.Solve()
    
    print('Total cost:', assignment.OptimalCost())
    
    for i in range(assignment.NumNodes()):
        print('Worker {wrk} assigned to task {tsk} with cost {cst}'.format(
        wrk=i, tsk=assignment.RightMate(i), cst=assignment.AssignmentCost(i)))
        
if __name__ == "__main__":
    main()


Total cost: 276
Worker 0 assigned to task 1 with cost 76
Worker 1 assigned to task 3 with cost 65
Worker 2 assigned to task 2 with cost 90
Worker 3 assigned to task 0 with cost 45


In [14]:
from ortools.graph import pywrapgraph

def conjure_data():
    d = {'start_nodes':[0, 0, 0, 0] + [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4] + [5, 6, 7, 8],
         'end_nodes':  [1, 2, 3, 4] + [5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8] + [9, 9, 9, 9],
         'capacities': [1, 1, 1, 1] + [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] + [1, 1, 1, 1 ],
         'costs':[0, 0, 0, 0] + [90, 76, 75, 70, 35, 85, 55, 65, 125, 95, 90, 105, 45, 110, 95, 115] + [0, 0, 0, 0],
         'supplies': [4, 0, 0, 0, 0, 0, 0, 0, 0, -4],
         'source': 0,
         'sink': 9,
         'tasks': 4}
    
    return(d)

def main():
    
    data = conjure_data()
    start_nodes = data['start_nodes']
    end_nodes = data['end_nodes']
    capacities = data['capacities']
    costs = data['costs']
    supplies = data['supplies']
    source = data['source']
    sink = data['sink']
    tasks = data['tasks']
    n_edges = len(start_nodes)
    n_nodes = len(supplies)
    
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()
    
    for i in range(n_edges):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                                  capacities[i], costs[i])
        
    for i in range(n_nodes):
        min_cost_flow.SetNodeSupply(i, supplies[i])
        
    min_cost_flow.Solve()
    print('total cost', min_cost_flow.OptimalCost())
    
    for arc in range(min_cost_flow.NumArcs()):
        if min_cost_flow.Tail(arc)!=source and min_cost_flow.Head(arc)!=sink:
            if min_cost_flow.Flow(arc) > 0:
                print('Worder {wrk} assigned to task {tsk}, with cost {cst}'.format(
                wrk=min_cost_flow.Tail(arc),
                tsk=min_cost_flow.Head(arc),
                cst=min_cost_flow.UnitCost(arc)))

if __name__ == '__main__':
    main()            
    

total cost 265
Worder 1 assigned to task 8, with cost 70
Worder 2 assigned to task 7, with cost 55
Worder 3 assigned to task 6, with cost 95
Worder 4 assigned to task 5, with cost 45


### with teams

In [50]:
from ortools.graph import pywrapgraph
import time

def main():
    
    ## data part
    team_A = [1, 3, 5]
    team_B = [2, 4, 6]

    start_nodes = ([0, 0]  + [11, 11, 11] + [12, 12, 12] +
                   [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6] +
                   [7, 8, 9, 10])
    end_nodes =   ([11, 12] + team_A + team_B +
                   [7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10] +
                   [13, 13, 13, 13])
    capacities =  ([2, 2] + [1, 1, 1] + [1, 1, 1] +
                   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] +
                   [1, 1, 1, 1])
    costs      =  ([0, 0] + [0, 0, 0] + [0, 0, 0] +
                   [90, 76, 75, 70, 35, 85, 55, 65, 125, 95, 90, 105, 45, 110, 95, 115, 60, 105,
                    80, 75, 45, 65, 110, 95] + [0, 0, 0, 0])
    
    # Define an array of supplies at each node.
    supplies = [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4]
    source = 0
    sink = 13
    n_arcs = len(capacities)
    n_nodes = len(supplies)
    
    mcf = pywrapgraph.SimpleMinCostFlow()
    
    for i in range(n_arcs):
        mcf.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                         capacities[i], costs[i])

    for i in range(n_nodes):
        mcf.SetNodeSupply(i, supplies[i])
        
    mcf.Solve()
    print('total cost', mcf.OptimalCost())
          
    
    for arc in range(mcf.NumArcs()):
        if mcf.Tail(arc)!=source and mcf.Head(arc)!=sink and mcf.Tail(arc)!=11 and mcf.Tail(arc)!=12:
            if mcf.Flow(arc) > 0:
                print('Worker {wrk} assigned to task {tsk}, with cost {cst}'.format(
                wrk=mcf.Tail(arc),
                tsk=mcf.Head(arc),
                cst=mcf.UnitCost(arc)))

if __name__ == '__main__':
    start_time = time.clock()
    main()
    print("Time =", (time.clock() - start_time)*1000, "miliseconds")

total cost 250
Worker 1 assigned to task 9, with cost 75
Worker 2 assigned to task 7, with cost 35
Worker 5 assigned to task 10, with cost 75
Worker 6 assigned to task 8, with cost 65
Time = 2.1380000000004173 miliseconds


### Solving as Mix integer programming

In [47]:
from ortools.linear_solver import pywraplp

def main():
       
    #data part
    cost = [[90, 76, 75, 70],
            [35, 85, 55, 65],
            [125, 95, 90, 105],
            [45, 110, 95, 115],
            [60, 105, 80, 75],
            [45, 65, 110, 95]]
    team1 = [0, 2, 4]
    team2 = [1, 3, 5]
    team_max = 2
    n_workers = len(cost)
    n_tasks = len(cost[1])
    
    solver = pywraplp.Solver('filipe',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # the decision variable X - a dictionary of solver boolean variables
    x = {}
    for wrk in range(n_workers):
        for tsk in range(n_tasks):
            x[(wrk, tsk)] = solver.BoolVar('x_wrk={wrk}_tsk={tsk}'.format(wrk=wrk, tsk=tsk))
    
    # build the objective function: 
    solver.Minimize(solver.Sum([cost[wrk][tsk] * x[(wrk, tsk)] \
                                for wrk in range(n_workers) \
                                for tsk in range(n_tasks) ]))
    
    
    ### Constriants
    
    # for each wroker the sum of tasks is <= 1
    for wrk in range(n_workers):
        solver.Add(solver.Sum([x[(wrk, tsk)] for tsk in range(n_tasks)]) <= 1 )
    
    # each task gets allocated
    # for each tasks the sum of the allocation across the workers must be 1
    for tsk in range(n_tasks):
        solver.Add(solver.Sum([x[(wrk, tsk)] for wrk in range(n_workers)]) == 1 )
    
    # no more than 2 tasks per team
    solver.Add(solver.Sum([x[(wrk, tsk)] for tsk in range(n_tasks) for wrk in team1]) <= team_max )
    solver.Add(solver.Sum([x[(wrk, tsk)] for tsk in range(n_tasks) for wrk in team2]) <= team_max )
    
    sol = solver.Solve()
    
    print('Total cost = ', solver.Objective().Value())
    for wrk in range(n_workers):
        for tsk in range(n_tasks):
            if x[(wrk, tsk)].solution_value() > 0:
                print('Worker {wrk} assigned to task {tsk}.  Cost = {cst}'.format(
                    wrk=wrk, tsk=tsk, cst=cost[wrk][tsk]))

    print("Time = ", solver.WallTime(), " milliseconds")
    
if __name__ == '__main__':
    main()
    

Total cost =  250.0
Worker 0 assigned to task 2.  Cost = 75
Worker 1 assigned to task 0.  Cost = 35
Worker 4 assigned to task 3.  Cost = 75
Worker 5 assigned to task 1.  Cost = 65
Time =  11  milliseconds
